In [28]:
import numpy as np

In [34]:
def cartesian_product(*arrays):
    """
    Calculate the cartesian product of a list of input arrays.
    
    Parameters:
        arrays: a parameter list of 1d arrays

    Returns:
        ndarray of shape (N, D): a matrix with columns D equal to number of input arrays and 
        rows N equal to the product of number of elements in input arrays
    """
    # create a grid where each point is a combination of Chebyshev nodes across each dimension
    mg = np.meshgrid(*arrays, indexing='ij')
    st = np.stack(mg)
    st = np.transpose(st, list(range(1,len(st.shape))) + [0])
    return np.reshape(st, (-1, len(mg)))

def chebyshev_transform(x, a = -1.0, b = 1.0):
    """
    Transform input x from standard domain [-1, 1] to general
    hyper-rectangular domain.

    Parameters:
        x : 1d array or 2d array on the standard domain [-1, 1]
        a (float or 1d array) :
            lower boundary of domain, if a is array then we require a.shape[0]
            equal to x.shape[-1]
        b (float or 1d array):
            upper boundary of domain, if b is array then we require b.shape[0]
            equal to x.shape[-1]
    
    Returns:
        1d array or 2d array of shape x.shape    
    """
    return a + 0.5 * (b - a) * (x + 1.0)

def chebyshev_inverse_transform(y, a = -1.0, b = 1.0):
    """
    Transform input y from general hyper-rectangular domain to
    standard domain [-1, 1].

    Parameters
    ----------
    y : 1d array or 2d array on general hyper-rectangular domain
    a : float or 1d array
        lower boundary of domain, if a is array then we require a.shape[0]
        equal to y.shape[-1]
    b : float or 1d array
        upper boundary of domain, if b is array then we require b.shape[0]
        equal to y.shape[-1]
    
    Returns
    -------
    1d array or 2d array of shape y.shape    
    """
    return 2 / (b-a) * (y-a) - 1.0


def chebyshev_points(degrees):
    """
    Calculate the Chebyshev points (of second kind) used for interpolation.

    Parameters
    ----------
    degrees : list of int
        each entry represents the maximum polynomial degree per dimension,

    Returns
    -------
    list of 1d arrays


    Chebyshev points of second represent the extrema of Chebyshev polynomials
    of first kind.
    """
    
    return [ np.cos(np.pi * np.linspace(0, 1, n+1)) for n in degrees]


def chebyshev_multi_points(degrees):
    """
    Calculate multivariate Chebyshev points on a standard domain [-1,1].
    
    Parameters:
    degrees (list of int): each entry represents the maximum polynomial degree per dimension,
        len(degrees) corresponds to the number of dimensions
        
    Returns:
    2d array of shape (N,D): number of rows N equals product over all (N_d + 1) where N_d is
        the maximum polynomial degree in dimension d (d=1...D) and D is
        the number of dimensions of the tensor.
    """
    points = chebyshev_points(degrees)
    multi_points = cartesian_product(*points)
    
    return multi_points

def chebyshev_polynomials(x, max_degree):
    """
    Calculate the Chebyshev polynomials T_0(x), ..., T_N(x) for up to maximum degree N.

    Parameters:
        x : float or 1d array for interpolation x is assumed to be in [-1, 1] (element-wise)

    Returns: 
        1d array if input is float or 2d array if input is array. First dimension represents maximum degree N.
    """
    T = [ 1.0 + 0*x ]
    if max_degree==0:
        return np.array(T)
    T += [ x ]
    if max_degree==1:
        return np.array(T)
    for d in range(2, max_degree+1):
        T += [ 2*x*T[-1] - T[-2] ]
        
    return np.array(T)

def chebyshev_batch_call(C, x):
    """
    Calculate
    z = [...[C * T(x_D)] * ...] * T(x_1)
    for a tensor C and input points x.
    T(x_d) are Chebyshev polynomials to degree N_d.

    Parameters:
        C (ndarray): a tensor with suitable shape specifying the maximum polynomial degrees per dimension.
        x (ndarray): a (N,D) matrix where first dimension N represents batch size and
            second dimension D represents number of dimensions of tensor.
        
    Returns:
        1d array of shape (N,)

    This is the basic operation for interpolation.

    Re-shaping is to ensure proper broadcast in matmul.
    """
    degrees = [ d-1 for d in C.shape ]
    assert len(x.shape)==2
    assert x.shape[1]==len(degrees)
    res = np.reshape(C, [1] + list(C.shape))
    for xi, Nd in zip(reversed(x.T), reversed(degrees)):
        T = chebyshev_polynomials(xi, Nd)
        T = np.reshape(T.T, [T.shape[1]] + [1]*(len(res.shape)-3) + [T.shape[0]] + [1])
        if len(res.shape)==2: # last iteration
            res = np.reshape(res, [res.shape[0]]+[1]+[res.shape[1]])
        res = np.matmul(res, T)
        res = np.reshape(res, res.shape[:-1])
    
    return np.reshape(res, (-1))

def chebyshev_coefficients(degrees, multi_points, values):
    """
    Calculate coefficients of Chebyshev basis functions.

    Parameters:
        degrees (list of int): each entry represents the maximum polynomial degree per dimension,
            len(degrees) corresponds to the number of dimensions
        multi_points (2d array): multivariate Chebyshev points on a standard domain [-1,1]
        values (1d array): the target function values for each D-dimensional multivariate
            Chebyshev point on the transformed domain of the tagret function.
        
    Returns: 
        ndarray of shape (degrees + 1)
    """
    assert len(multi_points.shape) == 2
    assert len(values.shape) == 1
    assert multi_points.shape[0] == np.prod([n+1 for n in degrees])
    assert multi_points.shape[0] == values.shape[0]
    assert multi_points.shape[1] == len(degrees)
    #
    idx_list = [ np.arange(n+1) for n in degrees]
    multi_idx = cartesian_product(*idx_list)
    inner_node_factor = np.sum(
        (0<multi_idx) & (multi_idx<np.array(degrees)),
        axis = 1
    )
    values_adj = values * 0.5**(len(degrees) - inner_node_factor)
    values_adj = np.reshape(values_adj, [n+1 for n in degrees])  # as tensor
    multi_coeff = chebyshev_batch_call(values_adj, multi_points)
    multi_coeff *= 2**inner_node_factor / np.prod(degrees)
    coeff = np.reshape(multi_coeff, [n+1 for n in degrees]) # as tensor 
    
    return coeff

def chebyshev_interpolation(y, coeff, a = -1.0, b = 1.0):
    """
    Calcuate multivariate Chebyshev interpolation.

    Parameters:
        y (2d array): Inputs of shape (N, D) where N is batch size and D is the number
            of dimensios. Input is assumed from general hyper-rectangular domain
        coeff (ndarray): calibrated Chebyshev tensor coefficients
        a (float or 1d array): lower boundary of domain, if a is array then we require a.shape[0]
            equal to y.shape[-1]
        b (float or 1d array): upper boundary of domain, if b is array then we require b.shape[0]
            equal to y.shape[-1]
    
    Returns: 
        1darray of shape (N,)
    """
    x = chebyshev_inverse_transform(y, a, b)
    
    return chebyshev_batch_call(coeff, x)

In [52]:
# Example parameters for the GBM
S_0 = 100  # Initial stock price
mu = 0.05  # Drift term
sigma = 0.2  # Volatility
delta_t = 1/252  # Time step in years (daily step in a 252-day trading year)
N_MC = 10000  # Number of Monte Carlo simulations

# Nodal points corresponding to Chebyshev nodes
nodal_points = np.array([-0.5,0,0.5])

def simulate_gbm_end_points(S_0, mu, sigma, delta_t, N_MC):
    # Simulate end points directly using the GBM formula
    Z = np.random.randn(N_MC)  # Random draws from standard normal distribution
    end_points = S_0 * np.exp((mu - 0.5 * sigma**2) * delta_t + sigma * np.sqrt(delta_t) * Z)
    return end_points

def monte_carlo_generalized_moments_chebyshev(nodal_points, N_MC, delta_t, mu, sigma):
    N = len(nodal_points)
    Gamma = np.zeros((N, N))
    
    for k, x_k in enumerate(nodal_points):
        end_points = simulate_gbm_end_points(x_k, mu, sigma, delta_t, N_MC)
        for j in range(N):
            # The Chebyshev polynomials are defined on the domain [-1, 1], hence we rescale the simulation end points
            scaled_end_points = 2 * (end_points - nodal_points[0]) / (nodal_points[-1] - nodal_points[0]) - 1
            p_j_values = np.cos(j * np.arccos(scaled_end_points))
            Gamma[k, j] = np.mean(p_j_values)

# Calculate the generalized moments matrix using the adjusted Monte Carlo simulation
Gamma = monte_carlo_generalized_moments_chebyshev(nodal_points, N_MC, delta_t, mu, sigma)
Gamma

/var/folders/3j/22yv_sj10t96slp1b8b6z1zw0000gn/T/ipykernel_22746/1734803278.py:26: RuntimeWarning: invalid value encountered in arccos
  p_j_values = np.cos(j * np.arccos(scaled_end_points))


In [53]:
# Define the number of nodes (degree of Chebyshev polynomial) for each dimension
degrees = [3]  # Replace with actual degrees

# Calculate the Chebyshev points
chebyshev_nodes = chebyshev_multi_points(degrees)

# Define the domain boundaries for each dimension
domain_boundaries = [(30,40)]  # Replace with actual boundaries

In [42]:
chebyshev_nodes

array([[ 1. ],
       [ 0.5],
       [-0.5],
       [-1. ]])

# Exploring the methods

In [35]:
# Example arrays
array1 = np.array([1, 2])
array2 = np.array([4, 5])
array3 = np.array([6, 7])

# Calculate the cartesian product of the example arrays
cartesian_product_result = cartesian_product(array1, array2, array3)
cartesian_product_result

array([[1, 4, 6],
       [1, 4, 7],
       [1, 5, 6],
       [1, 5, 7],
       [2, 4, 6],
       [2, 4, 7],
       [2, 5, 6],
       [2, 5, 7]])

In [36]:
z = np.array([-1, -0.5, 0, 0.5, 1])  # univariate chebyshev nodes on standardized domain
x_lower, x_upper = 0, 10  # domain bounds on the general domain

# Apply the transformations
transformed_z = chebyshev_transform(z, x_lower, x_upper)
transformed_z

array([ 0. ,  2.5,  5. ,  7.5, 10. ])

In [37]:
# Example usage
x_values = np.linspace(-1, 1, 5)  # Example x values in the range [-1, 1]
max_degree = 5  # Compute Chebyshev polynomials up to degree 5

# Compute the Chebyshev polynomials for the given x_values
chebyshev_poly_values = chebyshev_polynomials(x_values, max_degree)

chebyshev_poly_values  # Display the computed Chebyshev polynomial

array([[ 1. ,  1. ,  1. ,  1. ,  1. ],
       [-1. , -0.5,  0. ,  0.5,  1. ],
       [ 1. , -0.5, -1. , -0.5,  1. ],
       [-1. ,  1. , -0. , -1. ,  1. ],
       [ 1. , -0.5,  1. , -0.5,  1. ],
       [-1. , -0.5,  0. ,  0.5,  1. ]])

In [38]:
# Example usage:
# Create a tensor C with random coefficients
C = np.random.rand(3, 3, 3)  # Example tensor for a 3D Chebyshev series up to degree 2 in each dimension

# Create an array of points x where each row is a point in 3D space
x = np.random.rand(5, 3) * 2 - 1  # 5 points in the range [-1, 1] for each dimension

# Call the function
z = chebyshev_batch_call(C, x)
z

array([0.2036025 , 0.60786277, 0.6191806 , 1.36291899, 0.24367263])